# Pipeline for Creating Design Matrix for Regression
This code is a pipeline that generates and saves a pandas dataframe for a specific travel route in downtown Toronto before being fed for regression model predictions

In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
# matplotlib.rcParams['figure.dpi'] = 144
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
import datetime 
import pandas_profiling
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn.ensemble import RandomForestRegressor
from pydoc import help
from scipy.stats.stats import pearsonr
import forestci as fci
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

Failed to import duecredit due to No module named 'duecredit'


### Setup Parameters

In [2]:
data_type='bt'
segment='E_D'
start_date='2014-04-10'
end_date='2017-01-31'

resamp_method='H'
df = pd.read_csv('/Users/tung-linwu/Desktop/Insight/data/compiled_data/'+segment+'_'+data_type+'_combined.csv')

### Anomally Z score filter

In [26]:
df['timeInSeconds Z'] = abs(df['timeInSeconds'] - df['timeInSeconds'].mean())/df['timeInSeconds'].std(ddof=0)
df = df[df['timeInSeconds Z'] < 3]

### Assign Timestamps

In [27]:
format2 = '%Y-%m-%dT%H:%M:%S-%f'
df['Datetime'] = pd.to_datetime(df['updated'], format=format2)
df = df.set_index(pd.DatetimeIndex(df['Datetime']))
df=df.loc[start_date:end_date]
df1 = df[['timeInSeconds']]
df1=df1.loc[start_date:end_date]
daily = df1.resample(resamp_method).mean()

### Add Days of Week to Design Matrix

In [28]:
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
for i in range(7):
    daily[days[i]] = (daily.index.dayofweek == i).astype(float)

### Add Hour of Day and Annual Count to Design Matrix

In [8]:
daily['hourfloat2']=daily.index.hour
daily['dummy'] = daily['hourfloat2'].apply(lambda x: -4 if x >= 4  else 20)
daily['hourfloat'] = daily['dummy'] + daily['hourfloat2']
daily = daily.drop(labels='hourfloat2', axis=1)
daily = daily.drop(labels='dummy', axis=1)
daily['x']=np.sin(2.*np.pi*daily.hourfloat/24.)
daily['y']=np.cos(2.*np.pi*daily.hourfloat/24.)
daily['annual'] = (daily.index - daily.index[0]).days/365

### Add Weather to Design Matrix

In [9]:
df2 = pd.read_csv('/Users/tung-linwu/Desktop/Insight/data/compiled_data/Weather_combined.csv')
df2['Snow'] = np.where(df2['Weather']=='Snow', 1, 0)
format3 = '%Y-%m-%d %H:%M'
df2['Datetime'] = pd.to_datetime(df2['Date/Time'], format=format3)
df2 = df2.set_index(pd.DatetimeIndex(df2['Datetime']))
df2 = df2[['Temp (°C)','Snow','Visibility (km)']]
df2 = df2.resample(resamp_method).mean()
df_w=df2.sort_values(by=['Datetime'])
df_w=df_w.loc[start_date:end_date]
daily=pd.concat((daily, df_w), axis=1)
daily.head()

### Add Holidays to Design Matrix

In [12]:
holiday = pd.read_csv('/Users/tung-linwu/Desktop/Insight/data/compiled_data/holiday_combined.csv')
holiday = holiday.set_index(pd.DatetimeIndex(holiday['Datetime']))
holiday = holiday[['Event Value']]
holiday=holiday.loc[start_date:end_date]
holiday=holiday.resample('1H').pad(limit=23)
daily=pd.concat((daily, holiday), axis=1)
daily.rename(columns={'Event Value': 'holiday'}, inplace=True)

### Add NBA to Design Matrix

In [13]:
nba = pd.read_csv('/Users/tung-linwu/Desktop/Insight/data/compiled_data/nba_combined.csv')
nba=nba[nba['Home/Neutral'].str.contains("Toronto Raptor")]
format5 = '%m%d%y'
nba['Datetime'] = pd.to_datetime(nba['Date']+' '+nba['Start (ET)'], infer_datetime_format=True)
nba = nba.set_index(pd.DatetimeIndex(nba['Datetime']))
nba = nba[['Attend.']]
nba2=nba.resample('1H').backfill(limit=5)
nba2=nba2.dropna()
nba2 = nba2.resample(resamp_method).sum()
nba2=nba2.loc[start_date:end_date]
daily=pd.concat((daily, nba2), axis=1)
daily.rename(columns={'Attend.': 'NBA'}, inplace=True)

### Add MLB to Design Matrix

In [16]:
mlb = pd.read_csv('/Users/tung-linwu/Desktop/Insight/data/compiled_data/mlb_combined_time.csv')
mlb = mlb.set_index(pd.DatetimeIndex(mlb['Datetime']))
mlb = mlb[['attendance']]
mlb2=mlb.resample('1H').backfill(limit=5)
mlb2=mlb2.dropna()
mlb2 = mlb2.resample(resamp_method).sum()
mlb2=mlb2.loc[start_date:end_date]
daily=pd.concat((daily, mlb2), axis=1)
daily.rename(columns={'attendance': 'MLB'}, inplace=True)

### Add NHL to Design Matrix

In [31]:
nhl = pd.read_csv('/Users/tung-linwu/Desktop/Insight/data/compiled_data/nhl_combined_time.csv')
nhl = nhl.set_index(pd.DatetimeIndex(nhl['Datetime']))
nhl = nhl[['attendance']]
nhl2=nhl.resample('1H').backfill(limit=5)
nhl2=nhl2.dropna()

nhl2 = nhl2.resample(resamp_method).sum()
nhl2=nhl2.loc[start_date:end_date]
daily=pd.concat((daily, nhl2), axis=1)
daily.rename(columns={'attendance': 'NHL'}, inplace=True)

### Add events at Scotiabank Centre to Design Matrix

In [18]:
events = pd.read_csv('/Users/tung-linwu/Desktop/Insight/data/events/table-2_edit.txt')
events['Datetime'] = pd.to_datetime(events['Dates'], infer_datetime_format=True)
events = events.set_index(pd.DatetimeIndex(events['Datetime']))
events['Events'] = 1
events = events[['Events']]
events=events.loc[start_date:end_date]
events=events.resample('1H').pad(limit=24)
daily=pd.concat((daily, events), axis=1)

### Replacing backfilled NaNs with 0s

In [19]:
# Fill in NaNs with 0s for sports/Events and whether to convert to binary
daily['NBA'].fillna(0, inplace=True)
daily['MLB'].fillna(0, inplace=True)
daily['NHL'].fillna(0, inplace=True)
daily['Events'].fillna(0, inplace=True)
daily['holiday'].fillna(0, inplace=True)

### Replacing Attendances to Binary

In [ ]:
daily['NBA'] = np.where(daily['NBA']>0, 1, 0)
daily['MLB'] = np.where(daily['MLB']>0, 1, 0)
daily['NHL'] = np.where(daily['NHL']>0, 1, 0)

### Dropping Rows with NaNs and Saving

In [29]:
daily=daily.dropna()
daily.to_csv('/Users/tung-linwu/Desktop/Insight/data/design_matrix/D_matrix_' +segment+'_'+data_type+ '.csv')